In [1]:
library(ggplot2)
library(ggmap)
library(sp)
library(maptools)
library(rgdal)
library(rgeos)
library(RColorBrewer)
library(dplyr)
library(leaflet)

Warning message:
: package ‘ggplot2’ was built under R version 3.2.4Checking rgeos availability: TRUE
rgdal: version: 1.1-8, (SVN revision 616)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.11.3, released 2015/09/16
 Path to GDAL shared files: /usr/local/Cellar/gdal/1.11.3_1/share/gdal
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-2 
rgeos version: 0.3-19, (SVN revision 524)
 GEOS runtime version: 3.5.0-CAPI-1.9.0 r4084 
 Linking to sp version: 1.2-2 
 Polygon checking: TRUE 


Attaching package: ‘dplyr’

The following objects are masked from ‘package:rgeos’:

    intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
crime = read.csv('train.csv')
str(crime)

'data.frame':	878049 obs. of  9 variables:
 $ Dates     : Factor w/ 389257 levels "2003-01-06 00:01:00",..: 389257 389257 389256 389255 389255 389255 389255 389255 389254 389254 ...
 $ Category  : Factor w/ 39 levels "ARSON","ASSAULT",..: 38 22 22 17 17 17 37 37 17 17 ...
 $ Descript  : Factor w/ 879 levels "ABANDONMENT OF CHILD",..: 867 811 811 405 405 407 740 740 405 405 ...
 $ DayOfWeek : Factor w/ 7 levels "Friday","Monday",..: 7 7 7 7 7 7 7 7 7 7 ...
 $ PdDistrict: Factor w/ 10 levels "BAYVIEW","CENTRAL",..: 5 5 5 5 6 3 3 1 7 2 ...
 $ Resolution: Factor w/ 17 levels "ARREST, BOOKED",..: 1 1 1 12 12 12 12 12 12 12 ...
 $ Address   : Factor w/ 23228 levels "0 Block of  HARRISON ST",..: 19791 19791 22698 4267 1844 1506 13323 18055 11385 17659 ...
 $ X         : num  -122 -122 -122 -122 -122 ...
 $ Y         : num  37.8 37.8 37.8 37.8 37.8 ...


In [3]:
dim(crime[crime$Y==90.0,])

[1] 67  9

In [4]:
crime = crime[crime$Y!=90.0,]

In [5]:
#convert to spatialpointsDataFrame with X/Y coordinate and WGS84 datum
coords = SpatialPoints(crime[,c('X','Y')])
crime_spatial = SpatialPointsDataFrame(coords,crime)
proj4string(crime_spatial) = CRS('+proj=longlat +ellps=WGS84')

In [6]:
str(crime_spatial)

Formal class 'SpatialPointsDataFrame' [package "sp"] with 5 slots
  ..@ data       :'data.frame':	877982 obs. of  9 variables:
  .. ..$ Dates     : Factor w/ 389257 levels "2003-01-06 00:01:00",..: 389257 389257 389256 389255 389255 389255 389255 389255 389254 389254 ...
  .. ..$ Category  : Factor w/ 39 levels "ARSON","ASSAULT",..: 38 22 22 17 17 17 37 37 17 17 ...
  .. ..$ Descript  : Factor w/ 879 levels "ABANDONMENT OF CHILD",..: 867 811 811 405 405 407 740 740 405 405 ...
  .. ..$ DayOfWeek : Factor w/ 7 levels "Friday","Monday",..: 7 7 7 7 7 7 7 7 7 7 ...
  .. ..$ PdDistrict: Factor w/ 10 levels "BAYVIEW","CENTRAL",..: 5 5 5 5 6 3 3 1 7 2 ...
  .. ..$ Resolution: Factor w/ 17 levels "ARREST, BOOKED",..: 1 1 1 12 12 12 12 12 12 12 ...
  .. ..$ Address   : Factor w/ 23228 levels "0 Block of  HARRISON ST",..: 19791 19791 22698 4267 1844 1506 13323 18055 11385 17659 ...
  .. ..$ X         : num [1:877982] -122 -122 -122 -122 -122 ...
  .. ..$ Y         : num [1:877982] 37.8 37.8 37.8

In [7]:
bbox(crime_spatial)

,min,max
X,-122.5136,-122.3649
Y,37.70788,37.81998


In [8]:
ca_shp = readShapeSpatial('tl_2015_06_tract/tl_2015_06_tract.shp',proj4string = CRS('+proj=longlat +datum=WGS84'))

In [ ]:
plot(ca_shp,xlim=c(-122.5136, -122.3649), ylim=c(37.70788,37.81998),col='grey',axes=TRUE)

plot(crime_spatial, pch=21, b='red',cex=.5,add=TRUE)

In [9]:
#use ggmap
ca_shp_df = fortify(ca_shp)

Regions defined for each Polygons


In [10]:
locations = c(left = -122.5222, 
                bottom = 37.7073, 
                right = -122.3481,
                top = 37.8381)
map_data = get_map(location=locations, zoom=12, source='osm',color='bw')

In [11]:
sf_map = ggmap(map_data,extent='device')
sf_map1 = sf_map + geom_point(aes(x=X,y=Y),data=crime,alpha=0.5,color='red',size=0.5)

In [15]:
png('sfmap_1.png')
sf_map1
dev.off()

pdf 
  2